In [8]:
from datetime import datetime
from meteostat import Point, Daily
from matplotlib import pyplot as plt
import analyzerdf
import services
import logging
import logging.config
import json

In [9]:
config = None
with open('config/logger_config.json', 'r') as file:
    config = json.load(file)
logging.config.dictConfig(config)

In [10]:
samara = Point(53.2, 50.15, 116)
time_start = datetime(2015, 1, 1)
time_end = datetime(2017, 12, 31)

time_start2 = datetime(2010, 1, 1)
time_end2 = datetime(2010, 12, 31)

In [11]:
samara_temperature_df = Daily(samara, time_start, time_end).fetch()
samara_temperature_df.dropna(axis='columns', how='all', inplace=True)
samara_temperature_df_analyzer = analyzerdf.Analyzer(samara_temperature_df,
                                     [samara._lat, samara._lon, samara._alt])

samara_temperature_df2 = Daily(samara, time_start2, time_end2).fetch()
samara_temperature_df2.dropna(axis='columns', how='all', inplace=True)
samara_temperature_df_analyzer2 = analyzerdf.Analyzer(samara_temperature_df2,
                                     [samara._lat, samara._lon, samara._alt])

In [12]:
service = services.Service(samara_temperature_df_analyzer)

In [15]:
service2 = services.Service(samara_temperature_df_analyzer2)

In [ ]:
service.start()

2024-11-19 17:07:50,705 MainProcess services.service INFO     Сервис запустился


2024-11-19 17:07:50,792 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:07:57,815 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:08:04,872 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:08:11,912 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:08:18,957 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:08:26,004 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:08:33,027 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:08:40,089 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:08:47,133 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временно

In [16]:
service2.start()

2024-11-19 17:09:30,610 MainProcess services.service INFO     Сервис запустился


2024-11-19 17:09:30,697 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:09:37,746 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:09:44,800 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:09:51,840 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:09:58,880 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:10:05,935 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:10:12,985 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:10:20,064 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временного ряда
2024-11-19 17:10:27,090 MainProcess analyzerdf.analyzerdf.analyzer INFO     Найден дифференциал временно

In [7]:
service.stop()

2024-11-19 16:54:55,042 MainProcess services.service INFO     Сервис завершил работу


In [10]:
service2.stop()

2024-11-19 15:25:40,498 MainProcess services.service INFO     Сервис завершил работу


In [ ]:
analyzerdf.write_in_excel(samara_temperature_df_analyzer.data_frame, 
                          "standart df")

In [ ]:
df_extremums = samara_temperature_df_analyzer.get_extremums()
analyzerdf.write_in_excel(df_extremums, 'extremums')

In [ ]:
fig1, ax1 = plt.subplots()
ax1.plot(samara_temperature_df_analyzer.data_frame['tavg'])
for i in df_extremums['tavg'].index:
    ax1.scatter(x = i, y = df_extremums['tavg'].at[i], s = 5, c = 'r')
ax1.set_title('График температуры в Самаре')
ax1.set_ylabel('Значения средней температуры')
ax1.set_xlabel('Даты')

In [ ]:
df_rolling_meaning = samara_temperature_df_analyzer.get_rolling_meaning(30)
df_rolling_meaning.dropna(axis='rows', how='all', inplace=True)

df_rolling_meaning_analyzer = analyzerdf.Analyzer(df_rolling_meaning, [samara._lat, samara._lon, samara._alt])
analyzerdf.write_in_excel(df_rolling_meaning_analyzer.data_frame, "roll mean")

df_rolling_meaning_extremums = df_rolling_meaning_analyzer.get_extremums()
analyzerdf.write_in_excel(df_rolling_meaning_extremums, 
                          "extremums from roll mean")

In [ ]:
fig2, ax2 = plt.subplots()
ax2.plot(df_rolling_meaning_analyzer.data_frame['tavg'])
for i in df_rolling_meaning_extremums['tavg'].index:
    ax2.scatter(x = i, 
                y = df_rolling_meaning_extremums['tavg'].at[i], 
                s = 5, 
                c = 'r')
ax2.set_title('График скользящего среднего температуры в Самаре')
ax2.set_ylabel('Значения скользящего среднего с окном \n' + 
               'размера 30 средней температуры')
ax2.set_xlabel('Даты')

In [ ]:
df_rolling_meaning_diff = df_rolling_meaning_analyzer.get_differential()
analyzerdf.write_in_excel(df_rolling_meaning_diff, "diff from roll mean")

In [ ]:
fig3, ax3 = plt.subplots()
ax3.plot(df_rolling_meaning_diff['tavg'])
ax3.set_title('График дифференциала температуры в Самаре')
ax3.set_ylabel('Значения дифференциала от скользящего \n' + 
               'среднего размера 30 средней температуры')
ax3.set_xlabel('Даты')

In [ ]:
df_rolling_meaning_autocorr = df_rolling_meaning_analyzer.get_autocorrelation()
df_rolling_meaning_autocorr.dropna(axis='columns', how='all', inplace=True)
analyzerdf.write_in_excel(df_rolling_meaning_autocorr, 
                          "autocorr from roll mean")

In [ ]:
fig4, ax4 = plt.subplots()
ax4.plot(df_rolling_meaning_autocorr['tavg'])
ax4.set_title('График автокорелляции средней температуры в Самаре')
ax4.set_ylabel('Значения коэффициентов корреляции')
ax4.set_xlabel('Даты')